In [1]:
print("hello world")

hello world


In [2]:
import fileinput
file1 = 'csvs/MesesLSM.csv'
file2 = 'csvs/MesesLSM_faces.csv'
a = []
with fileinput.input(files=(file2,
    file1)) as f:
    #f.nextfile()
    print(f.readline())
    print(f.fileno())
    print(f.filename())
    print(f.nextfile())
    

0.0,0.0,594.0,96.0,798.0,300.0

4
csvs/MesesLSM_faces.csv
None


In [3]:
import pandas as pd

file1 = 'csvs/MesesLSM.csv'

###
lector = pd.read_csv(file1, header = 1)

lector.head()

,Tiempo inicial - hh:mm:ss.ms,Tiempo final - hh:mm:ss.ms,Nombre,ConfiguracionMano,Movimiento,Ubicación,Orientacion,PuntodeContacto,PlanodeArticulacion,ComponentesNomanuales,Unnamed: 10
0,00:00:00.910,00:00:02.690,Enero,"Letra ""E"" del abecedario dactilologico\t",Semicircular,Entre barbilla y pecho,Mano lateral con meñique al a vista,Sin contacto,Se gira 1 vez y se regresa al lugar de origen....,Sin componentes,NaN
1,00:00:04.100,00:00:06.140,Febrero,"Letra ""F"" del abecedario dactilogico",Semicircular,Entre barbilla y pecho,Mano lateral con Meñique a la vista,Sin contacto,Se gira 1 vez y se regresa origen. 2 veces.,Sin comoponentes,NaN
2,00:00:08.970,00:00:10.910,Marzo,"Letra ""M"" del abecedario dactilologico",Circular,"Cara, al lado de oreja de mano dominante","Con la ""M"" girar el circulo hacia adelante",Sin contacto,Se pueden hacer 1 o 2 movimientos circulares,Sin componentes,NaN
3,00:00:12.050,00:00:14.090,Marzo (2),"Se deletrea abreviado ""M"" ""R"" ""Z"" ""O""",Sin movimiento.,Entre barbilla y pecho,Palma abajo dorso arriba,Sin contacto,Sin plano,Sin compomentes,NaN
4,00:00:15.740,00:00:17.750,Abril,"Letra ""A"" del abecedario dactilologico",Circular,"Cara, al lado de oreja de mano dominante","Con la ""A"" girar el circulo hacia adelante",Sin contacto,Se pueden hacer una o dos movimientos,Sin componentes,NaN


In [4]:
lector["Tiempo inicial - hh:mm:ss.ms"]

0     00:00:00.910
1     00:00:04.100
2     00:00:08.970
3     00:00:12.050
4     00:00:15.740
5     00:00:20.035
6     00:00:23.920
7     00:00:27.220
8     00:00:29.230
9     00:00:32.285
10    00:00:34.830
11    00:00:37.290
12    00:00:39.725
13    00:00:41.870
Name: Tiempo inicial - hh:mm:ss.ms, dtype: object

In [5]:
from datetime import datetime, timedelta, time
format = '%H:%M:%S.%f'
#columna = time(second = 0, tzinfo=None, fold=0)
#columna = timedelta(microseconds=0)
columna = []

i = 0
for row in lector["Tiempo inicial - hh:mm:ss.ms"]:
    # .isoformat(timespec = 'milliseconds')
    print(datetime.strptime(row, format).isoformat(timespec = 'milliseconds'))
    columna.append(datetime.strptime(row, format).isoformat(timespec = 'milliseconds'))
    i+=1
    #print(type(row), row)
lector["Tiempo inicial - hh:mm:ss.ms"] = columna


1900-01-01T00:00:00.910
1900-01-01T00:00:04.100
1900-01-01T00:00:08.970
1900-01-01T00:00:12.050
1900-01-01T00:00:15.740
1900-01-01T00:00:20.035
1900-01-01T00:00:23.920
1900-01-01T00:00:27.220
1900-01-01T00:00:29.230
1900-01-01T00:00:32.285
1900-01-01T00:00:34.830
1900-01-01T00:00:37.290
1900-01-01T00:00:39.725
1900-01-01T00:00:41.870


In [6]:
ti = []
tf = []
for time_i in lector["Tiempo inicial - hh:mm:ss.ms"]:
    ti.append(time_i[-12:])

for time_f in lector["Tiempo final - hh:mm:ss.ms"]:
    tf.append(time_f[-12:])

lector["Tiempo inicial - hh:mm:ss.ms"] = ti

lector["Tiempo final - hh:mm:ss.ms"] = tf
  

# ENTONCES PRIMERO CORTO LOS VIDEOS CON MOVIEPY, LUEGO EXTRAIGO CARAS CON OPENCV Y TAMBIÉN KEYPOINTS CON OPENPOSE, ESTO PARA CADA VIDEO RECORTADO CON MOVIEPY


In [7]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

#fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()
fgbg = cv2.createBackgroundSubtractorMOG2()

while(1):
    ret, frame = cap.read()

    fgmask = fgbg.apply(frame)

    cv2.imshow('frame',fgmask)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [8]:
import cv2

movie_path = 'Videos/MesesLSM.mp4'

capture = cv2.VideoCapture(0)
num_frames = int(capture.get(7))

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

while True:
    ret, frame = capture.read()
    img = frame
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w] 


    cv2.imshow("Demo", img)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

capture.release()
cv.destroyAllWindows()


NameError: name 'cv' is not defined

In [ ]:
import datetime

num_frames = 1200
freq = 60 / FPS

dti = pd.date_range('2020-01-01', periods = num_frames, freq = 'ms')